In [157]:
pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [158]:
import pandas as pd
import numpy as np
#NLP Environment setup 
from collections import Counter 

#Plotting 
import squarify
import matplotlib.pyplot as plt 
import seaborn as sns 

#NLP Libraries 
import re
from nltk.stem import PorterStemmer
import spacy
from spacy.tokenizer import Tokenizer#NLP Environment setup 
from collections import Counter 

#Plotting 
import squarify
import matplotlib.pyplot as plt 
import seaborn as sns 

#NLP Libraries 
import re
from nltk.stem import PorterStemmer
import spacy
from spacy.tokenizer import Tokenizer


In [159]:
import gensim
import os
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

from gensim.models.ldamulticore import LdaMulticore
from gensim.parsing.preprocessing import preprocess_string, strip_tags, strip_punctuation, strip_numeric, stem_text
from gensim.parsing.preprocessing import strip_multiple_whitespaces, strip_non_alphanum, remove_stopwords, strip_short

In [160]:
df = pd.read_csv('/Users/noahpovis/Desktop/Buildweek22/Build Week 2 /DX.csv')


In [161]:
train, val, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])


In [162]:
train.shape

(16278, 18)

In [163]:
val.shape

(5426, 18)

In [164]:
test.shape

(5427, 18)

In [165]:
def clean(i):
    i = i.copy()
    i.drop_duplicates(subset=['Organization Name','Funding Type'], inplace = True)
    i.drop_duplicates(keep='first', inplace= True)
    i['Raised Series A'] = np.where(i['Organization Name'].duplicated(), 'Yes', 'No')
    i.drop_duplicates(subset='Organization Name', keep='last', inplace= True)
    i['Money Raised Currency (in USD)'] = pd.to_numeric(i['Money Raised Currency (in USD)'], errors='coerce')
    i['Total Funding Amount Currency (in USD)'] = pd.to_numeric(i['Total Funding Amount Currency (in USD)'], errors='coerce')
    i['Announced Date'] = pd.to_datetime(i['Announced Date'], errors='coerce')
    i['Total Funding Amount Currency (in USD)'] = pd.to_numeric(i['Total Funding Amount Currency (in USD)'], errors='coerce')
    i  = i[i['Funding Type'] != 'Series A'] 
    return i 



In [166]:
train = clean(train)
val = clean(val)
test = clean(test)


In [167]:
train['Raised Series A'].value_counts()

No     12410
Yes      182
Name: Raised Series A, dtype: int64

In [168]:
train.shape , val.shape, test.shape

((12592, 19), (4493, 19), (4572, 19))

In [169]:
train['Investor Names'].isnull().sum()

3622

In [170]:
def split_columns(text):
    text = text.copy()
    # new data frame with split value columns 
    new = text["Organization Industries"].str.split(",", n = 7, expand = True) 
    # making separate first name column from new data frame 
    text["Primary Industry"]= new[0] 
    # making separate last name column from new data frame 
    text["Sub_Ind"]= new[1] 
    text["Sub_Ind2"]= new[2]
    text["Sub_Ind3"]= new[3]
    text["Sub_Ind4"]= new[4]
    text["Sub_Ind5"]= new[5]
    text["Sub_Ind6"]= new[6]
    text["Sub_Ind7"]= new[7]
    # Dropping old Name columns 
    text.drop(columns =["Organization Industries"], inplace = True)

    #same thing but for investor column 
    new1 = text["Investor Names"].str.split(",", n = 11, expand = True) 
    # making separate first name column from new data frame 
    text["Investor1"]= new1[0] 
    # making separate last name column from new data frame 
    text["Investor1"]= new1[1] 
    text["Investor2"]= new1[2]
    text["Investor3"]= new1[3]
    text["Investor4"]= new1[4]
    text["Investor5"]= new1[5]
    text["Investor6"]= new1[6]
    text["Investor7"]= new1[7]
    text["Investor8"]= new1[8]
    text["Investor9"]= new1[9]
    text["Investor10"]= new1[10]
    text["Investor11"]= new1[11]
    # Dropping old Name columns 
    text.drop(columns =["Investor Names"], inplace = True)
    text.drop(columns =["Lead Investors"], inplace = True)



    return text 



In [171]:
train = split_columns(train)
val = split_columns(val)
test = split_columns(test)


In [172]:
train.columns

Index(['Transaction Name', 'Transaction Name URL', 'Organization Name',
       'Organization Name URL', 'Funding Type', 'Money Raised',
       'Money Raised Currency', 'Money Raised Currency (in USD)',
       'Announced Date', 'Total Funding Amount',
       'Total Funding Amount Currency',
       'Total Funding Amount Currency (in USD)', 'Organization Description',
       'Organization Location', 'Number of Investors', 'Raised Series A',
       'Primary Industry', 'Sub_Ind', 'Sub_Ind2', 'Sub_Ind3', 'Sub_Ind4',
       'Sub_Ind5', 'Sub_Ind6', 'Sub_Ind7', 'Investor1', 'Investor2',
       'Investor3', 'Investor4', 'Investor5', 'Investor6', 'Investor7',
       'Investor8', 'Investor9', 'Investor10', 'Investor11'],
      dtype='object')

In [173]:
train.isnull().sum()

Transaction Name                              0
Transaction Name URL                          0
Organization Name                             0
Organization Name URL                         0
Funding Type                                  0
Money Raised                               4206
Money Raised Currency                      4203
Money Raised Currency (in USD)             4207
Announced Date                                1
Total Funding Amount                       3126
Total Funding Amount Currency              3124
Total Funding Amount Currency (in USD)     3127
Organization Description                      0
Organization Location                       282
Number of Investors                        3622
Raised Series A                               0
Primary Industry                            493
Sub_Ind                                    2262
Sub_Ind2                                   4608
Sub_Ind3                                   7754
Sub_Ind4                                

In [174]:
# Defining 'preprocessing_filter' function
def preprocessing_filter(text):
    '''
    Cleans text for font size, whitespace,
    removing stop words, and stripping non-alpha 
    numeric characters
    '''
    # Custom Preprocessing Filter
    custom_filters = [lambda x: x.lower(),          # Lowercase
                      strip_multiple_whitespaces,   # Remove numbers
                      remove_stopwords,             # Remove stopwords
                      strip_non_alphanum,           # Remove non-alpha numeric characters
                     ]
    
    
    # TEST / CHECK: Preprocess with 'custom_filters'
    processed_text = preprocess_string(text, custom_filters)
    
    # Returning 'processed_text'
    return processed_text

In [175]:
def function_apply(k):
    k["tokenized_Description"] = k["Organization Description"].apply(preprocessing_filter)
    k.drop(columns =["Organization Description"], inplace = True)
    print(k.shape)

    return k 

In [176]:
train = function_apply(train)
val = function_apply(val)
test = function_apply(test)

(12592, 35)
(4493, 35)
(4572, 35)


In [177]:
train['Raised Series A'].value_counts()

No     12410
Yes      182
Name: Raised Series A, dtype: int64

In [178]:
val['Raised Series A'].value_counts()

No     4473
Yes      20
Name: Raised Series A, dtype: int64

In [179]:
test['Raised Series A'].value_counts()

No     4549
Yes      23
Name: Raised Series A, dtype: int64

In [183]:
def corpora(d):
    #'Dictionary' Representation of all the words in our corpus
    id2word = corpora.Dictionary(d["tokenized_Description"])
    #Removing extreme values from the dataset
    id2word.filter_extremes(no_below=5, no_above=0.95)
    #bag of words(bow) representation of our corpus
    corpus = [id2word.doc2bow(text) for text in d["tokenized_Description"]]
    return d 

In [184]:
train = corpora(train)
val = corpora(val)
test = corpora(test)





AttributeError: 'function' object has no attribute 'Dictionary'